In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.layers import *
from tensorflow.keras import Model

%matplotlib inline

In [2]:
(X,Y),(x,y) = tf.keras.datasets.mnist.load_data()

X = X.reshape(-1,784) / 255
x = x.reshape(-1,784) / 255

Y = tf.keras.utils.to_categorical(Y)
y = tf.keras.utils.to_categorical(y)

In [5]:
inp = Input(shape=(784,),name="input")

d1 = Dense(128,activation="relu",name="dense_1")(inp)
d2 = Dense(64,activation="relu",name="dense_2")(d1)
d3 = Dense(32,activation="relu",name="dense_3")(d2)

out = Dense(10,activation="softmax",name="out")(d3)

model = Model([inp],[out])

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
out (Dense)                  (None, 10)                330       
Total params: 111,146
Trainable params: 111,146
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])

In [8]:
model.fit(X,Y,epochs=1,batch_size=32,validation_data=(x,y))

1875/1875 [==============================] - 5s 3ms/step - loss: 0.2585 - accuracy: 0.9235 - val_loss: 0.1325 - val_accuracy: 0.9599


In [15]:
!mkdir "../save_models/MNIST_DENSE/"

A subdirectory or file ../save_models/MNIST_DENSE/ already exists.


In [16]:
open("../save_models/MNIST_DENSE/model.json","w+").write(model.to_json())
model.save_weights("../save_models/MNIST_DENSE/model")